# 5.6. GPU
- 使用nvidia-smi命令来查看显卡信息。
- 在PyTorch中，每个数组都有一个设备（device）， 我们通常将其称为上下文（context）。 默认情况下，所有变量和相关的计算都分配给CPU。 有时上下文可能是GPU。 当我们跨多个服务器部署作业时，事情会变得更加棘手。
    - 通过智能地将数组分配给上下文， 我们可以最大限度地减少在设备之间传输数据的时间。 例如，当在带有GPU的服务器上训练神经网络时， 我们通常希望模型的参数在GPU上。
- 要运行此部分中的程序，至少需要两个GPU。 注意，对于大多数桌面计算机来说，这可能是奢侈的，但在云中很容易获得。 例如，你可以使用AWS EC2的多GPU实例。
- 本书的其他章节大都不需要多个GPU， 而本节只是为了展示数据如何在不同的设备之间传递。

## 5.6.1 计算设备
我们可以指定用于存储和计算的设备，如CPU和GPU。 默认情况下，张量是在内存中创建的，然后使用CPU计算它。

In [1]:
import torch
from torch import nn

"""
在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。
应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。 然而，gpu设备只代表一个卡和相应的显存。
如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第块GPU（i从0开始）。
另外，cuda:0和cuda是等价的。
"""
torch.device('cpu'),torch.device('cuda')
# torch.device('cuda1')  如果有第二块显卡

(device(type='cpu'), device(type='cuda'))

In [2]:
# 查询可用gpu的数量。
torch.cuda.device_count()

1

In [3]:
# 这两个函数允许我们在不存在所需所有GPU的情况下运行代码。
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices=[torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 5.6.2. 张量与GPU

In [4]:
# 查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

"""
需要注意的是，无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。 例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。
"""

x=torch.tensor([1,2,3])
x.device


device(type='cpu')

### 存储在GPU上

In [5]:
"""
在创建张量时指定存储设备,在第一个gpu上创建张量变量X
"""
# X = torch.ones(2, 3, device=try_gpu())
X=torch.ones(2,3,device=torch.device('cuda'))
X

"""
在第二个GPU上创建一个随机张量Y。
"""
Y=torch.rand(2,3,device=try_gpu(1))
Y

tensor([[0.2008, 0.5060, 0.8973],
        [0.4483, 0.3379, 0.3223]])

### 复制
- 简单地X加上Y，因为这会导致异常， 运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。
- 由于Y位于第二个GPU上，所以我们需要将X移到那里， 然后才能执行相加运算。

![avatar](../img/5_2.png)

In [6]:
Z = X.cuda(1) # 将X复制到Z上
X,Z
"""
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')
"""

Y+Z # 现在数据在同一个GPU上（Z和Y都在），我们可以将它们相加。
"""
tensor([[1.1379, 1.4532, 1.9869],
        [1.4779, 1.7426, 1.9622]], device='cuda:1')
"""

Z.cuda(1) is Z # Z.cuda(1)将返回Z，而不会复制并分配新内存。
"""
True
"""

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

### 旁注
单个GPU相对运行速度快。 但是在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。 这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收）， 然后才能继续进行更多的操作。


当我们打印张量或将张量转换为NumPy格式时， 如果数据不在内存中，框架会首先将其复制到内存中， 这会导致额外的传输开销。 更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

## 5.6.3. 神经网络与GPU
类似地，神经网络模型可以指定设备。

In [12]:
"""
将模型参数放在GPU上。
"""
net = nn.Sequential(nn.Linear(3,1))
net=net.to(device=try_gpu())

# 当输入为GPU上的张量时，模型将在同一GPU上计算结果。
net(X)


tensor([[-0.7334],
        [-0.7334]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
# 确认模型参数存储在同一个GPU上。只要所有的数据和参数都在同一个设备上， 我们就可以有效地学习模型。
net[0].weight.data.device

device(type='cuda', index=0)

## 5.6.4. 小结
- 我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。
- 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。
- 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。